In [1]:
import pandas as pd

data = pd.read_csv('~/ctgan/data/annual_GA_stdDev_bal_3class_36.csv')
data.head()

,State,City,Date Sown,Variety,Date of Cut,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%),Standard Deviation,Average Yield,Number of Deviations,Class
0,Georgia,Athens,10/10/2007,BaraWet 501,12/5/2008,340,2.99,422,1,3545.32,413.282,18.354044,12.543772,24.636933,0.129465,1.625241,4.706667,-1.056254,1
1,Georgia,Athens,10/10/2007,GA-505,12/5/2008,340,3.08,422,1,3545.32,413.282,18.354044,12.543772,24.636933,0.129465,1.625241,4.706667,-1.000877,1
2,Georgia,Midville,11/1/2007,GA-505,8/14/2009,226,2.40,652,267,3400.10,740.670,21.585771,16.097508,27.881281,0.191288,1.625241,4.706667,-1.419277,1
3,Georgia,Midville,11/1/2007,Hybri Force 600,8/14/2009,226,2.37,652,267,3400.10,740.670,21.585771,16.097508,27.881281,0.191288,1.625241,4.706667,-1.437735,1
4,Georgia,Midville,11/1/2007,BaraWet 501,8/14/2009,226,2.40,652,267,3400.10,740.670,21.585771,16.097508,27.881281,0.191288,1.475449,4.568452,-1.469689,1


In [2]:
#from sdv.tabular import CTGAN

#model = CTGAN()
#model.fit(data)

In [3]:
#new_data = model.sample(1000)

In [4]:
#new_data.to_csv('data/non_annualized_synth3_GA_KY_1K.csv')
#new_data.head()

In [5]:
# get aggregate data
#targetDataLoc = 'data/GA_only_063022.csv'
#aggDataLoc = 'data/synth1_GA_only_063022.csv'

aggDf = data #pd.read_csv(aggDataLoc)
#aggDf = aggDf.drop("Unnamed: 0",axis=1)
targetDf = data #pd.read_csv(targetDataLoc)
#targetDf = targetDf.drop("Unnamed: 0",axis=1)

In [6]:
############## imports
# general
import statistics
import datetime
#from sklearn.externals import joblib # save and load models
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score  
# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# models
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVC  # support vector machines classification
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsClassifier # k-nearest neightbors for classification
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier # adaboost for classification
# saving models
# from sklearn.externals import joblib
import joblib

# import the API
APILoc = 'API/'

import sys
sys.path.insert(0, APILoc)

from API import *

In [7]:
# filter out the features that will not be used by the machine learning models

# the features to keep:
# xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Time Since Last Harvest (Days)", "Total Radiation (MJ/m^2)",
#                "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
#                  "Avg Soil Moisture (%)", "Day Length (hrs)"], "Percent Cover (%)"]

xColumnsToKeep = ["Total Radiation (MJ/m^2)",
               "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
                 "Avg Soil Moisture (%)"]


#xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)", "Total Rainfall (mm)"]

# the target to keep
yColumnsToKeep = ["Class"]

# get a dataframe containing the features and the targets
xDf = aggDf[xColumnsToKeep]
test_xDf = aggDf[xColumnsToKeep]
yDf = aggDf[yColumnsToKeep]
test_yDf = aggDf[yColumnsToKeep]

# reset the index
xDf = xDf.reset_index(drop=True)
yDf = yDf.reset_index(drop=True)
test_xDf = test_xDf.reset_index(drop=True)
test_yDf = test_yDf.reset_index(drop=True)

pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)

xCols = list(xDf)

In [8]:
# hide the warnings because training the neural network caues lots of warnings.
import warnings
warnings.filterwarnings('ignore')

# make the parameter grids for sklearn's gridsearchcv
rfParamGrid = {
        'model__n_estimators': [5, 10, 25, 50, 100], # Number of estimators
        'model__max_depth': [5, 10, 15, 20], # Maximum depth of the tree
        'model__criterion': ["gini"]
    }
knnParamGrid ={
        'model__n_neighbors':[2,5,10],
        'model__weights': ['uniform', 'distance'],
        'model__leaf_size': [5, 10, 30, 50]    
    }
svcParamGrid = {
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__C': [0.1, 1.0, 5.0, 10.0],
        'model__gamma': ["scale", "auto"],
        'model__degree': [2,3,4,5]
    }
nnParamGrid = {
        'model__hidden_layer_sizes':[(3), (5), (10), (3,3), (5,5), (7,7)],
        'model__solver': ['sgd', 'adam'],
        'model__learning_rate' : ['constant', 'invscaling', 'adaptive'],
        'model__learning_rate_init': [0.1, 0.01, 0.001]      
    }

logRegParamGrid = {}

bayesParamGrid={
        'model__n_iter':[100,300,500]
    }

dtParamGrid = {
    'model__criterion': ['gini'],
    'model__max_depth': [5,10,25,50,100]
    }

aModelList = [#(RandomForestClassifier(), rfParamGrid, "rfTup.pkl"),
              (KNeighborsClassifier(), knnParamGrid, "knnTup.pkl"),
              (SVC(), svcParamGrid, "svrTup.pkl"),
             #(MLPRegressor(), nnParamGrid, "nnTup.pkl")],
             (LogisticRegression(), logRegParamGrid, "linRegTup.pkl")]#,
             #(BayesianRidge(), bayesParamGrid, "bayesTup.pkl"),
             #(DecisionTreeClassifier(), dtParamGrid, "dtTup.pkl")]

N = 2
workingDir = 'working_dir'
numFeatures = 6 # 11

In [9]:
saveMLResults(test_xDf, test_yDf, N, xDf, yDf, aModelList, workingDir, numFeatures, printResults=True)

truth:  [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3]
predictions:  [1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 3, 3, 3, 3, 3, 3, 1, 1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3]
model:  knnTup
Avg MAE:  500.0
Avg R squared:  0.617
Best MAE:  444.444
Best R squared:  0.691
Parameters of the best model:  {'model__leaf_size': 5, 'model__n_neighbors': 2, 'model__weights': 'uniform'}
Features selected by best model:  ['Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moisture (%)']
Accuracy:  0.75
f1 score:  0.7777777777777777
mcc:  0.6822429906542056
 
test results on our test data: 
results:
MAE:  0.19444444444444445
R sq:  0.7083333333333333
accuracy score for DA/TDA:  0.8055555555555556
truth:  [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3]
predictions:  [1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 2, 3, 3